In [37]:
import string
import autograd.numpy as np
from nltk.corpus import stopwords
import operator
import nltk

In [38]:
### This code computes the counts c(k) and c(k,w). I use dict to store the counts for class and word

stemmer = nltk.wordnet.WordNetLemmatizer() 
stop_words=set(stopwords.words('english'))

class_count = {} ## a dict to store c(k)
class_word_count = {} ## a dict to store c(k,w)
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/train') as f_train:
    for line in f_train:
        ## remove all the punctuations and stopwords
        line_lt = [w for w in line.split() if w not in string.punctuation if w not in stop_words] 
        ## remove the affixes
        line_list = []
        line_list.append(line_lt[0])
        for item in line_lt[1:]:
            line_list.append(stemmer.lemmatize(item))   ## use Wordnet Stemmer to remove affix
        ## Now line_list is a list containing all the words in one document
        
        ## Calculate the c(k)
        if line_list[0] not in class_count :
            class_count [line_list[0]] = 1
        else:
            class_count[line_list[0]] += 1
        
        ## Calculate the c(k,w)
        if line_list[0] not in class_word_count:
            class_word_count[line_list[0]]={}
            for w in line_list[1:]:
                if w not in class_word_count[line_list[0]]:
                    class_word_count[line_list[0]][w] = 1
                else: 
                    class_word_count[line_list[0]][w] += 1
        else:
            for w in line_list[1:]:
                if w not in class_word_count[line_list[0]]:
                    class_word_count[line_list[0]][w] = 1
                else: 
                    class_word_count[line_list[0]][w] += 1

In [39]:
### Counts for Trump and Clinton for words country and president
c_trump = class_count['trump']
c_clinton = class_count['clinton']
c_trump_president = class_word_count['trump']['president']
c_trump_country = class_word_count['trump']['country']
c_clinton_president = class_word_count['clinton']['president']
c_clinton_country = class_word_count['trump']['country']

print("c(trump): {} ".format(c_trump))
print ("c(clinton): {}".format(c_clinton))
print ("c(trump, president): {}".format(c_trump_president))
print ("c(trump,country): {}".format(c_trump_country))
print ("c(clinton,president): {}".format(c_clinton_president))
print ("c(clinton,country): {}".format(c_clinton_country))

c(trump): 637 
c(clinton): 455
c(trump, president): 41
c(trump,country): 174
c(clinton,president): 187
c(clinton,country): 174


In [40]:
### Calculate the probability of each class p(k)
total_count = 0
for i in class_count:
    total_count = total_count + class_count[i]

class_prob = {} ## p(k) stores in this dict

for i in class_count:
    class_prob[i]=class_count[i]/total_count

In [41]:
## Probability of trump and clinton
p_trump = class_prob['trump']
p_clinton = class_prob['clinton']

print ("p(trump): {}".format(p_trump))
print ("p(clinton): {}".format(p_clinton))

p(trump): 0.20183776932826364
p(clinton): 0.14416983523447402


In [42]:
### Calculate the probability p(w|k) with add-one smoothing

## Calculate the V
all_word_seen = []
for i in class_word_count:
    for j in class_word_count[i]:
        all_word_seen.append(j)
all_word_seen = set(all_word_seen)
size_seenword = len(all_word_seen)
V=size_seenword + 1
        
## Claculate p(w|k) 

class_word_prob = {} ##Stores p(w|k)
total_word_eachclass = {}

for i in class_word_count:
    class_word_prob[i]={}
    total_word_eachclass[i] = 0
    for j in class_word_count[i]:
        total_word_eachclass[i] = total_word_eachclass[i] + class_word_count[i][j]
    for k in class_word_count[i]:
        class_word_prob[i][k] = (class_word_count[i][k]+1)/(total_word_eachclass[i]+V)  

In [43]:
## Probability of trump and clinton for words country and president
p_trump_country = class_word_prob['trump']['country']
p_trump_president = class_word_prob['trump']['president']
p_clinton_country = class_word_prob['clinton']['country']
p_clinton_president = class_word_prob['clinton']['president']

print ("p(country|trump): {}".format(p_trump_country))
print ("p(president|trump): {}".format(p_trump_president))
print ("p(country|clinton): {}".format(p_clinton_country))
print ("p(president|clinton): {}".format(p_clinton_president))

p(country|trump): 0.0073396804093444614
p(president|trump): 0.0017615232982426709
p(country|clinton): 0.003429218271407644
p(president|clinton): 0.0062591556798508455


In [44]:
## calculate the p(k/d) for the firtst document in dev file
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
    dev_1st_before_stemmer = [w for w in next(f_dev).split() if w not in string.punctuation if w not in stop_words]
    dev_1st = []
    dev_1st.append(dev_1st_before_stemmer[0])
    for item in dev_1st_before_stemmer[1:]:
        dev_1st.append(stemmer.lemmatize(item))

## calculat P(k/d)
    class_dev_1st_prob = {}
    class_dev_1st_normalized_prob = {}
    class_dev_1st_logprob = {}

    for i in class_word_prob:
        sum_log_pro_allword = 0
        for j in dev_1st[1:]:
            if j in class_word_prob[i]:
                sum_log_pro_allword = sum_log_pro_allword + np.log(class_word_prob[i][j])
            else:
                sum_log_pro_allword = sum_log_pro_allword + np.log(1/(total_word_eachclass[i]+V))
            
        class_dev_1st_logprob[i]= np.log(class_prob[i])+sum_log_pro_allword
    
## Deal with the underflow problem
## Before doing the np.exp() on log-prob, minus the largest log-prob for each class, then the largest log-prb is going to be 1

largest_log_prob = max(class_dev_1st_logprob.values())

for item in class_dev_1st_logprob:
    class_dev_1st_prob[item] = np.exp(class_dev_1st_logprob[item]-largest_log_prob)

total_prob = 0
for item in class_dev_1st_prob:
    total_prob = total_prob + class_dev_1st_prob[item]

for item in class_dev_1st_prob:
    class_dev_1st_normalized_prob[item] = class_dev_1st_prob[item]/total_prob

## check whether the total probability summed to be 1

total_normalized_prob = 0
for item in class_dev_1st_normalized_prob:
    print ("P({}|1st document in dev): {}".format(item,class_dev_1st_normalized_prob[item]))
    total_normalized_prob = total_normalized_prob + class_dev_1st_normalized_prob[item]
    
print ("sum of all the probability: {}".format(total_normalized_prob))
    
predict_class = [key for key,val in class_dev_1st_normalized_prob.items() if val == max(class_dev_1st_normalized_prob.values())]
print ("predicted speaker: {}".format(predict_class)) 

P(rubio|1st document in dev): 0.057742183296234026
P(huckabee|1st document in dev): 1.0761957001997873e-14
P(o'malley|1st document in dev): 6.591641421421866e-13
P(fiorina|1st document in dev): 2.4586610838097202e-14
P(clinton|1st document in dev): 0.9422377042069229
P(chafee|1st document in dev): 1.547888366297945e-23
P(paul|1st document in dev): 5.885568160330661e-10
P(carson|1st document in dev): 1.690131764574655e-09
P(webb|1st document in dev): 4.043591542783662e-14
P(cruz|1st document in dev): 1.5314081325768036e-05
P(trump|1st document in dev): 1.497867066347483e-07
P(kasich|1st document in dev): 3.09320144671935e-08
P(perry|1st document in dev): 3.2596431399191214e-28
P(sanders|1st document in dev): 1.9505681907308288e-07
P(walker|1st document in dev): 6.667568162998306e-13
P(bush|1st document in dev): 4.4067383972904645e-06
P(christie|1st document in dev): 1.3621489520720117e-08
sum of all the probability: 1.0000000000000002
predicted speaker: ['clinton']


In [45]:
## Function to predict the class
def predict_class(doc):
    class_prediction = {}
    class_logprob = {}
    for i in class_word_prob:
        sum_log_pro_allword = 0
        for j in doc[1:]:
            if j in class_word_prob[i]:
                sum_log_pro_allword = sum_log_pro_allword + np.log(class_word_prob[i][j])
            else:
                sum_log_pro_allword = sum_log_pro_allword + np.log(1/(total_word_eachclass[i]+V))
        class_logprob[i] =np.log(class_prob[i])+sum_log_pro_allword   
    
    largest_log_prob = max(class_logprob.values())
    
    for i in class_logprob:
        class_prediction[i]= np.exp(class_logprob[i]-largest_log_prob)        
    
    prediction = [key for key,val in class_prediction.items() if val == max(class_prediction.values())]
    return prediction

In [46]:
## Calculate the accuracy for dev file

dev_prediction = []
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
    for doc in f_dev:
        dev_doc_before_stemmer = [w for w in doc.split() if w not in string.punctuation if w not in stop_words]
        
        dev_doc = []
        dev_doc.append(dev_doc_before_stemmer[0])
        for item in dev_doc_before_stemmer[1:]:
            dev_doc.append(stemmer.lemmatize(item))
            
        dev_doc_pred = {}
        dev_doc_pred[dev_doc[0]] = predict_class(dev_doc)
        dev_prediction.append(dev_doc_pred)

count = 0 
for i in dev_prediction:
    for key, value in i.items():
        if key == value[0]:
            count +=1
print ('Prediction accuracy on the dev file is {}' .format(count/len(dev_prediction)))

Prediction accuracy on the dev file is 0.515


In [47]:
## Calculate the accuracy for test file

test_prediction = []
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/test') as f_test:
    for doc in f_test:
        test_doc_before_stemmer = [w for w in doc.split() if w not in string.punctuation if w not in stop_words]
        test_doc = []
        test_doc.append(test_doc_before_stemmer[0])
        for item in test_doc_before_stemmer[1:]:
            test_doc.append(stemmer.lemmatize(item))
        
        test_doc_pred = {}
        test_doc_pred[test_doc[0]] = predict_class(test_doc)
        test_prediction.append(test_doc_pred)

count = 0 
for i in test_prediction:
    for key, value in i.items():
        if key == value[0]:
            count +=1
print ("Prediction accuracy on the test file is {}".format(count/len(test_prediction)))

Prediction accuracy on the test file is 0.5375
